In [1]:
import os,datetime
import cv2
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import random
from pathlib import Path
'''
SEED_XML_DIR = '/data/darknet/python/xml_special/'
SEED_IMG_DIR = '../generated/images_replaced_bkg/'
GENE_IMG_DIR = '../generated/image_expand_obj/'
GENE_XML_DIR = '../generated/xml_expand_obj/'
'''
SEED_XML_DIR = '/data/wxjs/Changed_Annotations/'
SEED_IMG_DIR = '/data/wxjs/JPEGImages/'
GENE_IMG_DIR = '/data/wxjs/generated/image_expand_obj/'
GENE_XML_DIR = '/data/wxjs/generated/xml_expand_obj/'
# RT:RightTop 
# LB:LeftBottom 
# bbox: [xmin, xax, ymin, ymax]
def IOU(bbox_a, bbox_b):
    '''
    W = min(A.RT.x, B.RT.x) - max(A.LB.x, B.LB.x) 
    H = min(A.RT.y, B.RT.y) - max(A.LB.y, B.LB.y) 
    if W <= 0 or H <= 0: 
        return 0 
    SA = (A.RT.x - A.LB.x) * (A.RT.y - A.LB.y) 
    SB = (B.RT.x - B.LB.x) * (B.RT.y - B.LB.y) 
    cross = W * H return cross/(SA + SB - cross)
    '''
    W = min(bbox_a[1], bbox_b[1]) - max(bbox_a[0], bbox_b[0]) 
    H = min(bbox_a[3], bbox_b[3]) - max(bbox_a[2], bbox_b[2]) 
    if W <= 0 or H <= 0: 
        return 0
    SA = (bbox_a[1] - bbox_a[0]) * (bbox_a[3] - bbox_a[2]) 
    SB = (bbox_b[1] - bbox_b[0]) * (bbox_b[3] - bbox_b[2])  
    cross = W * H 
    return cross/(SA + SB - cross)

def get_obj_from_xml(xml):
    in_file = open(xml_name)
    tree=ET.parse(in_file)
    root = tree.getroot()
    return [obj for obj in root.iter('object')]

def get_obj_from_image_file(file, bbox):
    img = cv2.imread(file)
    img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
    print(img_obj.shape)
    return img_obj

def get_bboxes_from_etree(etree):
    root = tree.getroot()  
    objects = root.findall('object')
    bboxes = []
    for obj in objects:
        xmlbox = obj.find('bndbox')
        b = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
        bboxes.append(b)
    return bboxes
def past_to_background_from_image_file(file, bboxes, background_img_array, extend_spaces=0):
    
    img = cv2.imread(file)
    #img = img - 50
    if(img.shape != background_img_array.shape):
        print('shape not match')
        return
    #print(img.shape)
    #print(img)
    img_objs = []
    for bbox in bboxes:
        img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
        img_objs.append(img_obj)
    i = 0
    for bbox in bboxes:
        background_img_array[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])] = img_objs[i]
        i = i+1
    cv2.imwrite(GENE_IMG_DIR+file.split('/')[-1], background_img_array)
    return 
def generate_new_xmlobj(xmlobj_old, new_position, new_size):
    element_object = ET.Element('object')
    tag_name = ET.SubElement(element_object, 'name')
    tag_name.text = xmlobj_old.find('name').text

    tag_difficult = ET.SubElement(element_object, 'difficult')
    tag_difficult.text = xmlobj_old.find('difficult').text

    element_bndbox = ET.SubElement(element_object, 'bndbox')
    tag_xmin = ET.SubElement(element_bndbox, 'xmin')
    tag_ymin = ET.SubElement(element_bndbox, 'ymin')
    tag_xmax = ET.SubElement(element_bndbox, 'xmax')
    tag_ymax = ET.SubElement(element_bndbox, 'ymax')
    tag_xmin.text = str(new_position[0])
    tag_ymin.text = str(new_position[1])
    tag_xmax.text = str(new_position[0] + new_size[1])
    tag_ymax.text = str(new_position[1] + new_size[0])
    return element_object
    
def past_and_insert(img_obj, img_array, new_position, obj_element, etree):
    new_xmlobj = generate_new_xmlobj(obj_element, new_position, img_obj.shape[:2])
    new_xml_etree = insert_to_xml(new_xmlobj, etree)
    new_pil_img = past_obj_to_background(img_obj, img_array, new_position)
    return new_pil_img, new_xml_etree
    

def insert_to_xml(xml_obj, xml_etree):
    root = xml_etree.getroot()
    root.append(xml_obj)
    return xml_etree
    
def past_obj_to_background(img_obj, img_array, position = (200, 200)):
    img = Image.fromarray(img_array)
    img_obj = Image.fromarray(img_obj)
    img.paste(img_obj, position)
    return img
    
def get_cls_from_xmlobj(obj_element):    
    return obj_element.find('name').text

def get_bbox_from_xmlobj(obj_element):
    xmlbox = obj_element.find('bndbox')
    return [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), 
            float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
def generate_new_position(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    #print(new_position)
    return new_position
def generate_new_bbox(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    new_bbox = [random_x, random_x + img_obj_size[1], random_y, random_y + img_obj_size[0]]
    #print(new_bbox)
    return new_bbox
    
    
def inset_obj_to_an_image_and_xml(img_obj, img, obj_element, etree):
    new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    bboxes = get_bboxes_from_etree(etree)
    retry_times = 0
    while(not check_bbox(new_bbox, bboxes)):
        print('new_bbox not suitable, retry...')
        retry_times = retry_times + 1
        if(retry_times > 50):
            return False, False
        new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    #print('new_bbox succussful')
    new_pil_img, new_xml_etree = past_and_insert(img_obj, img, (new_bbox[0], new_bbox[2]), obj_element, etree)
    
    return new_pil_img, new_xml_etree

def check_bbox(new_bbox, bboxes):
    for bbox in bboxes:
        if(IOU(new_bbox, bbox) > 0.005):
            return False
        continue
    return True

def rotate_img(img, thealta):
    (h_, w_) = img.shape[:2]
    point_list = [(0, 0), (0, h_), (w_, h_), (w_, 0)]
    center = (w_ // 2, h_ // 2)
    roted_point_list = [((point[0]-center[0])*np.cos(np.pi*thealta/180) 
                         - (point[1]-center[1])*np.sin(np.pi*thealta/180) 
                         + center[0], (point[0]-center[0])*np.sin(np.pi*thealta/180) 
                         + (point[1]-center[1])*np.cos(np.pi*thealta/180) + center[1]) for point in point_list]
    #print(roted_point_list)

    temp = np.zeros((2, 4))
    temp[0] = [roted_point[0] for roted_point in roted_point_list]
    temp[1] = [roted_point[1] for roted_point in roted_point_list]
    (xmin, xmax, ymin, ymax) = (np.min(temp[0]), np.max(temp[0]), np.min(temp[1]), np.max(temp[1]))
    print('xmin: {0}, xmax: {1}, ymin: {2}, ymax: {3}'.format(xmin, xmax, ymin, ymax))

    roted_h, roted_w = ymax - ymin, xmax - xmin
    print('roted_h: {0}, roted_w: {1}'.format(roted_w, roted_h))
    
    top_bottom, left_right = int((roted_h - h_)/2), int((roted_w - w_)/2)
    print('top_bottom:{0}, left_right_:{1}'.format(top_bottom, left_right))
    padding = lambda arg : max(arg, 0)
    #dst = cv2.copyMakeBorder(img, padding(top_bottom), padding(top_bottom), padding(left_right), 
    #padding(left_right), cv2.BORDER_CONSTANT)
    dst = cv2.copyMakeBorder(img, padding(top_bottom), padding(top_bottom), padding(left_right), 
                             padding(left_right), cv2.BORDER_CONSTANT)
    
    (h, w) = dst.shape[:2]   
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, thealta, 1.0)
    rotated_img = cv2.warpAffine(dst, M, (w, h))
    if(top_bottom < 0):
        top_bottom = abs(top_bottom)
        rotated_img = rotated_img[top_bottom:-top_bottom, :, :]
    if(left_right < 0):
        left_right = abs(left_right)
        rotated_img = rotated_img[:, left_right:-left_right, :]
    return rotated_img
LOOP_TIMES = 1
seed_xml_names = os.listdir(SEED_XML_DIR)
#cls_to_rotate = ['slagcar_00', 'slagcar_01', 'slagcar_10', 'slagcar_11']
cls_to_rotate = ['slagcar_01','slagcar_11']
for i in range (LOOP_TIMES):
    for xml_name in seed_xml_names:
        img_name = SEED_IMG_DIR + xml_name[:-3] + 'jpg'
        xml_name = SEED_XML_DIR + xml_name
        print(img_name)
        print(xml_name)
        tree=ET.parse(xml_name)
        img = cv2.imread(img_name)
        objs = get_obj_from_xml(xml_name)
        flag_found = False
        i = 0
        for obj_element in objs:
            bbox = get_bbox_from_xmlobj(obj_element)           
            img_obj = get_obj_from_image_file(img_name, bbox)

            cls = get_cls_from_xmlobj(obj_element)

            if (cls in cls_to_rotate):
                print(cls)
                flag_found = True
                for thealta in [0]:
                    img_obj = rotate_img(img_obj, thealta)
                    new_pil_img, new_xml_etree = inset_obj_to_an_image_and_xml(img_obj, img, obj_element, tree)
                    if((new_pil_img == False) or (new_xml_etree == False)):
                        continue
                    tree = new_xml_etree
                    img = np.array(new_pil_img)
            '''
            else:
                new_pil_img, new_xml_etree = inset_obj_to_an_image_and_xml(img_obj, img, obj_element, tree)       
                tree = new_xml_etree
                img = np.array(new_pil_img)
            '''        
            '''
            new_pil_img, new_xml_etree = inset_obj_to_an_image_and_xml(img_obj, img, obj_element, tree)       
            tree = new_xml_etree
            img = np.array(new_pil_img)
            '''
        if(flag_found != True):
            continue
        ## save xml and img
        time_mark = datetime.datetime.now()
        time_str = time_mark.strftime("%Y%m%d%H%M%S_")
        new_pil_img = Image.fromarray(img[:,:,(2,1,0)])
        new_pil_img.save(GENE_IMG_DIR + time_str + img_name.split('/')[-1])
        tree.write(GENE_XML_DIR + time_str + xml_name.split('/')[-1])
        #break

/data/wxjs/JPEGImages/01c08702b7f7f562547e2a8718c8ab07.jpg
/data/wxjs/Changed_Annotations/01c08702b7f7f562547e2a8718c8ab07.xml
(659, 528, 3)
slagcar_01
xmin: 0.0, xmax: 528.0, ymin: 0.0, ymax: 659.0
roted_h: 528.0, roted_w: 659.0
top_bottom:0, left_right_:0
new_bbox not suitable, retry...
/data/wxjs/JPEGImages/022014a454e80e353bef2690ea813d87.jpg
/data/wxjs/Changed_Annotations/022014a454e80e353bef2690ea813d87.xml
(864, 644, 3)
slagcar_01
xmin: 0.0, xmax: 644.0, ymin: 0.0, ymax: 864.0
roted_h: 644.0, roted_w: 864.0
top_bottom:0, left_right_:0
/data/wxjs/JPEGImages/014f328da8e0a69c217fc10635c3f7a8.jpg
/data/wxjs/Changed_Annotations/014f328da8e0a69c217fc10635c3f7a8.xml
(998, 890, 3)
slagcar_11
xmin: 0.0, xmax: 890.0, ymin: 0.0, ymax: 998.0
roted_h: 890.0, roted_w: 998.0
top_bottom:0, left_right_:0
/data/wxjs/JPEGImages/0127d83c870c04c6db0682ed6dd27162.jpg
/data/wxjs/Changed_Annotations/0127d83c870c04c6db0682ed6dd27162.xml
(881, 574, 3)
slagcar_01
xmin: 0.0, xmax: 574.0, ymin: 0.0, ymax: 8

/data/wxjs/JPEGImages/00b3d7c4884e52899ddce21fa5d4eea7.jpg
/data/wxjs/Changed_Annotations/00b3d7c4884e52899ddce21fa5d4eea7.xml
(778, 553, 3)
slagcar_11
xmin: 0.0, xmax: 553.0, ymin: 0.0, ymax: 778.0
roted_h: 553.0, roted_w: 778.0
top_bottom:0, left_right_:0
(470, 357, 3)
slagcar_11
xmin: 0.0, xmax: 357.0, ymin: 0.0, ymax: 470.0
roted_h: 357.0, roted_w: 470.0
top_bottom:0, left_right_:0
/data/wxjs/JPEGImages/0232a676ea70ec4533d2d397129da511.jpg
/data/wxjs/Changed_Annotations/0232a676ea70ec4533d2d397129da511.xml
(952, 624, 3)
slagcar_11
xmin: 0.0, xmax: 624.0, ymin: 0.0, ymax: 952.0
roted_h: 624.0, roted_w: 952.0
top_bottom:0, left_right_:0
/data/wxjs/JPEGImages/024e2a12a1e16f32256427efa4a299bd.jpg
/data/wxjs/Changed_Annotations/024e2a12a1e16f32256427efa4a299bd.xml
(331, 243, 3)
slagcar_11
xmin: 0.0, xmax: 243.0, ymin: 0.0, ymax: 331.0
roted_h: 243.0, roted_w: 331.0
top_bottom:0, left_right_:0
new_bbox not suitable, retry...
new_bbox not suitable, retry...
(80, 64, 3)
slagcar_11
xmin: 0.

/data/wxjs/JPEGImages/010a2f7c617fdeddb09bec335df98e15.jpg
/data/wxjs/Changed_Annotations/010a2f7c617fdeddb09bec335df98e15.xml
(607, 477, 3)
slagcar_11
xmin: 0.0, xmax: 477.0, ymin: 0.0, ymax: 607.0
roted_h: 477.0, roted_w: 607.0
top_bottom:0, left_right_:0
/data/wxjs/JPEGImages/02ac13bf8f01cb1533f3a9ee1b4bbf19.jpg
/data/wxjs/Changed_Annotations/02ac13bf8f01cb1533f3a9ee1b4bbf19.xml
(1238, 926, 3)
slagcar_11
xmin: 0.0, xmax: 926.0, ymin: 0.0, ymax: 1238.0
roted_h: 926.0, roted_w: 1238.0
top_bottom:0, left_right_:0
/data/wxjs/JPEGImages/0267f10e5ec70fdd403e115d0311dfed.jpg
/data/wxjs/Changed_Annotations/0267f10e5ec70fdd403e115d0311dfed.xml
(929, 653, 3)
slagcar_11
xmin: 0.0, xmax: 653.0, ymin: 0.0, ymax: 929.0
roted_h: 653.0, roted_w: 929.0
top_bottom:0, left_right_:0
/data/wxjs/JPEGImages/01767776d90155853574bbf9b8f66484.jpg
/data/wxjs/Changed_Annotations/01767776d90155853574bbf9b8f66484.xml
(1012, 822, 3)
slagcar_11
xmin: 0.0, xmax: 822.0, ymin: 0.0, ymax: 1012.0
roted_h: 822.0, roted_

/data/wxjs/JPEGImages/022d0841346351c4c8f767242d6d5456.jpg
/data/wxjs/Changed_Annotations/022d0841346351c4c8f767242d6d5456.xml
(864, 662, 3)
slagcar_01
xmin: 0.0, xmax: 662.0, ymin: 0.0, ymax: 864.0
roted_h: 662.0, roted_w: 864.0
top_bottom:0, left_right_:0
/data/wxjs/JPEGImages/01017b4f5dc5d635199d9670afd949fe.jpg
/data/wxjs/Changed_Annotations/01017b4f5dc5d635199d9670afd949fe.xml
(734, 621, 3)
slagcar_01
xmin: 0.0, xmax: 621.0, ymin: 0.0, ymax: 734.0
roted_h: 621.0, roted_w: 734.0
top_bottom:0, left_right_:0
new_bbox not suitable, retry...
new_bbox not suitable, retry...
new_bbox not suitable, retry...
new_bbox not suitable, retry...
new_bbox not suitable, retry...
/data/wxjs/JPEGImages/01c672bdb18dc60de44be911d0f5a6a2.jpg
/data/wxjs/Changed_Annotations/01c672bdb18dc60de44be911d0f5a6a2.xml
(1039, 874, 3)
slagcar_11
xmin: 0.0, xmax: 874.0, ymin: 0.0, ymax: 1039.0
roted_h: 874.0, roted_w: 1039.0
top_bottom:0, left_right_:0
/data/wxjs/JPEGImages/025017ce46d063cb1f5db57fe2e48990.jpg
/dat